In [165]:
import sys
import re
import glob
import codecs
import unicodedata
import os
import wave
import contextlib
import numpy as np
import pycrfsuite
from sklearn.metrics import classification_report
from random import shuffle

In [124]:
'''f = codecs.open('train_pos.txt', 'r', encoding='utf-8')
p = codecs.open('train_normalized.txt', 'w', encoding='utf-8')
for line in f:
    line = unicodedata.normalize('NFC',line)
    line = re.sub(r"<[A-Z]{1,}\${0,}>", " ", line)
    line = re.sub(r"-", " ", line)
    line = re.sub(r"<", " ", line)
    line = re.sub(r">", " ", line)
    line = re.sub(r"#", " ", line)
    line = re.sub(r"/", " ", line)
    line = re.sub(r"\\", " ", line)
    line = re.sub(r",", " ", line)
    line = re.sub(u"،", " ", line)
    line = re.sub(u"؛", " ", line)
    line = re.sub(u"؟", " ", line)
    line = re.sub(r";", " ", line)
    line = re.sub(r"\"", " ", line)
    line = re.sub(u"۔", " ", line)
    line = re.sub(r"\.", " ", line)
    line = re.sub(r":", " ", line)
    line = re.sub(r"!", " ", line)
    line = re.sub(r"\'", " ", line)
    line = re.sub(r"%", u"٪", line)
    line = re.sub(u"ِ", "", line)
    line = re.sub(u"ُ", "", line)
    line = re.sub(u"َ", "", line)
    line = re.sub(u"ْ", "", line)
    line = re.sub(u"ٰ", "", line)
    line = re.sub(u"ً", "", line)
    line = re.sub(u"ّ", "", line)
    line = re.sub(u"\u200C{2,}", " ", line)
    line = re.sub(r"\s{2,}", " ", line)
    p.write(line.strip()+'\n')
f.close()
p.close()'''

In [129]:
'''f = codecs.open('train_normalized.txt', 'r', encoding='utf-8')
p = codecs.open('unique_characters.txt', 'w', encoding='utf-8')
characters = list()
for line in f:
    for letter in line:
        characters.append(letter)
f.close()
unique_characters = list(set(characters))
for i in unique_characters:
    p.write(i+'\n')
p.close()'''

In [2]:
'''f = codecs.open('train.txt', 'r', encoding='utf-8')
p = codecs.open('train_cleaned.txt', 'w', encoding='utf-8')
for line in f:
    line = re.sub(r"\s{2,}", " ", line)
    p.write(line.strip()+'\n')
f.close()
p.close()'''

In [166]:
'''# Randomly shuffle data
f = codecs.open('train_cleaned.txt', 'r', encoding='utf-8')
p = codecs.open('train_shuffle.txt', 'w', encoding='utf-8')
text = list()
for i in f:
    text.append(i.strip())
shuffle(text)
for i in text:
    p.write(i+'\n')
f.close()
p.close()'''

In [167]:
sentences = list()
f = codecs.open('train_shuffle.txt', 'r', encoding='utf-8')
for line in f:
    sentences.append(line)
f.close()

print len(sentences)
print sentences[100]

4325
موازنہ کے ذریعے پہلے نو مہینوں کے لیے زیروکس نے ڈالر ۱۲۹۷ بلین کے مالیہ پر ڈالر ۴۹۲ ملین یا ڈالر ۴۵۵ فی حصص کمائے



In [168]:
def prepare_sentence(sentence):
    sentence = re.sub(u"\s\u200C",u"\u200C", sentence)
    sentence = re.sub(u"\u200C\s",u"\u200C", sentence)
    
    lengths = [len(w) for w in sentence.split(" ")]
    positions = []

    next_pos = 0
    for length in lengths:
        next_pos = next_pos + length
        positions.append(next_pos)
    concatenated = sentence.replace(" ", "")

    chars = [c for c in concatenated]
    labels = [0 if not i in positions else 1 for i, c in enumerate(concatenated)]
    #Uncomment to incorporate ZWNJ
    for i, c in enumerate(chars):
        if c == u"\u200C" and i+1 < len(chars):
            labels[i+1] = 2
            del chars[i]
            del labels[i]
    
    return list(zip(chars, labels))

In [169]:
prepared_sentences = list()
for sentence in sentences:    
    prepared_sentences.append(prepare_sentence(sentence))
    
print([d for d in prepared_sentences[100]])

[(u'\u0645', 0), (u'\u0648', 0), (u'\u0627', 0), (u'\u0632', 0), (u'\u0646', 0), (u'\u06c1', 0), (u'\u06a9', 1), (u'\u06d2', 0), (u'\u0630', 1), (u'\u0631', 0), (u'\u06cc', 0), (u'\u0639', 0), (u'\u06d2', 0), (u'\u067e', 1), (u'\u06c1', 0), (u'\u0644', 0), (u'\u06d2', 0), (u'\u0646', 1), (u'\u0648', 0), (u'\u0645', 1), (u'\u06c1', 0), (u'\u06cc', 0), (u'\u0646', 0), (u'\u0648', 0), (u'\u06ba', 0), (u'\u06a9', 1), (u'\u06d2', 0), (u'\u0644', 1), (u'\u06cc', 0), (u'\u06d2', 0), (u'\u0632', 1), (u'\u06cc', 0), (u'\u0631', 0), (u'\u0648', 0), (u'\u06a9', 0), (u'\u0633', 0), (u'\u0646', 1), (u'\u06d2', 0), (u'\u0688', 1), (u'\u0627', 0), (u'\u0644', 0), (u'\u0631', 0), (u'\u06f1', 1), (u'\u06f2', 0), (u'\u06f9', 0), (u'\u06f7', 0), (u'\u0628', 1), (u'\u0644', 0), (u'\u06cc', 0), (u'\u0646', 0), (u'\u06a9', 1), (u'\u06d2', 0), (u'\u0645', 1), (u'\u0627', 0), (u'\u0644', 0), (u'\u06cc', 0), (u'\u06c1', 0), (u'\u067e', 1), (u'\u0631', 0), (u'\u0688', 1), (u'\u0627', 0), (u'\u0644', 0), (u'\u06

In [170]:
def checkdigit(char):
    digits = [u'۱',u'۲',u'۳',u'۴',u'۵',u'۶',u'۷',u'۸',u'۹',u'۰']
    if char in digits:
        return "true"
    return "false"
print checkdigit(u'ا')
print unicodedata.bidirectional(u'ا')

false
AL


In [171]:
def isnonjoiner(char):
    non_joiners = [u'ا', u'د', u'ڈ', u'ز', u'ذ', u'ر', u'ڑ', u'ژ', u'و', u'ے']
    if char in non_joiners:
        return "true"
    return "false"

In [205]:
def create_char_features(sentence, i):
    features = [
        'bias',
        'char=' + sentence[i][0],
        'char.isdigit=' + checkdigit(sentence[i][0]),
        'char.isnonjoiner=' + isnonjoiner(sentence[i][0]),
        'char.category=' + unicodedata.category(sentence[i][0]),
        'char.direction=' + unicodedata.bidirectional(sentence[i][0]),
    ]
    
    if i >= 1:
        features.extend([
            'char-1=' + sentence[i-1][0],
            'char-1:0=' + sentence[i-1][0] + sentence[i][0],
        ])
    else:
        features.append("BOS")
        
    if i >= 2:
        features.extend([
            'char-2=' + sentence[i-2][0],
            'char-2:0=' + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-2:-1=' + sentence[i-2][0] + sentence[i-1][0],
        ])
        
    if i >= 3:
        features.extend([
            'char-3:0=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0] + sentence[i][0],
            'char-3:-1=' + sentence[i-3][0] + sentence[i-2][0] + sentence[i-1][0],
        ])
        
        
    if i + 1 < len(sentence):
        features.extend([
            'char+1=' + sentence[i+1][0],
            'char:+1=' + sentence[i][0] + sentence[i+1][0],
        ])
    else:
        features.append("EOS")
        
    if i + 2 < len(sentence):
        features.extend([
            'char+2=' + sentence[i+2][0],
            'char:+2=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0],
            'char+1:+2=' + sentence[i+1][0] + sentence[i+2][0],
        ])
        
    if i + 3 < len(sentence):
        features.extend([
            'char:+3=' + sentence[i][0] + sentence[i+1][0] + sentence[i+2][0]+ sentence[i+3][0],
            'char+1:+3=' + sentence[i+1][0] + sentence[i+2][0] + sentence[i+3][0],
        ])
    
    return features



def create_sentence_features(prepared_sentence):
    return [create_char_features(prepared_sentence, i) for i in range(len(prepared_sentence))]

def create_sentence_labels(prepared_sentence):
    return [str(part[1]) for part in prepared_sentence]

In [206]:
X_train = [create_sentence_features(ps) for ps in prepared_sentences[:-825]]
y_train = [create_sentence_labels(ps)   for ps in prepared_sentences[:-825]]

In [207]:
print X_train[0]
print y_train[0]

[['bias', u'char=\u0645', 'char.isdigit=false', 'char.isnonjoiner=false', 'char.category=Lo', 'char.direction=AL', 'BOS', u'char+1=\u0633', u'char:+1=\u0645\u0633', u'char+2=\u0679', u'char:+2=\u0645\u0633\u0679', u'char+1:+2=\u0633\u0679', u'char:+3=\u0645\u0633\u0679\u0631', u'char+1:+3=\u0633\u0679\u0631'], ['bias', u'char=\u0633', 'char.isdigit=false', 'char.isnonjoiner=false', 'char.category=Lo', 'char.direction=AL', u'char-1=\u0645', u'char-1:0=\u0645\u0633', u'char+1=\u0679', u'char:+1=\u0633\u0679', u'char+2=\u0631', u'char:+2=\u0633\u0679\u0631', u'char+1:+2=\u0679\u0631', u'char:+3=\u0633\u0679\u0631\u0644', u'char+1:+3=\u0679\u0631\u0644'], ['bias', u'char=\u0679', 'char.isdigit=false', 'char.isnonjoiner=false', 'char.category=Lo', 'char.direction=AL', u'char-1=\u0633', u'char-1:0=\u0633\u0679', u'char-2=\u0645', u'char-2:0=\u0645\u0633\u0679', u'char-2:-1=\u0645\u0633', u'char+1=\u0631', u'char:+1=\u0679\u0631', u'char+2=\u0644', u'char:+2=\u0679\u0631\u0644', u'char+1:+2=\

In [208]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

In [209]:
trainer.set_params({
    'c1': 1.0, 
    'c2': 1e-3,
    'max_iterations': 60,
    'feature.possible_transitions': True
})

In [210]:
trainer.train('urdu-text-segmentation.crfsuite')

In [211]:
tagger = pycrfsuite.Tagger()
tagger.open('urdu-text-segmentation.crfsuite')

In [212]:
def segment_sentence(sentence):
    sentence = sentence.replace(" ", "")
    sentence = sentence.replace(u"\u200C", "") #Uncomment to incorporate ZWNJ
    prediction = tagger.tag(create_sentence_features(sentence))
    print prediction
    complete = ""
    for i, p in enumerate(prediction):
        if p == "1":
            complete += " " + sentence[i]
        #Uncomment to incorporate ZWNJ
        elif p == "2":
            complete += u"\u200C" + sentence[i]
        else:
            complete += sentence[i]
    return complete

In [213]:
print(segment_sentence(u"ان کو براہ‌راست موازنے کے قابل بناتے ہوئے ہر اشاریہ ۱۰۰ کو مساوی کرتے ہوتے ۱۹۶۹ کے بند ہونے پر واقع ہے"))

['0', '0', '1', '0', '1', '0', '0', '0', '2', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0']
ان کو براہ‌راست موازنے کے قابل بناتے ہوئے ہر اشاریہ ۱۰۰ کو مساوی کرتے ہوتے ۱۹۶۹ کے بند ہونے پر واقع ہے


In [216]:
#tp = 0.0
#fp = 0.0
#fn = 0.0
#n_correct = 0.0
#n_incorrect = 0.0
y_true = list()
y_pred = list()
for s in prepared_sentences[-825:]:
    prediction = tagger.tag(create_sentence_features(s))
    y_pred.extend(prediction)
    correct = create_sentence_labels(s)
    y_true.extend(correct)
#    zipped = list(zip(prediction, correct))
#    tp +=        len([_ for l, c in zipped if l == c and l == "1"])
#    fp +=        len([_ for l, c in zipped if l == "1" and c == "0"])
#    fn +=        len([_ for l, c in zipped if l == "0" and c == "1"])
#    n_incorrect += len([_ for l, c in zipped if l != c])
#    n_correct   += len([_ for l, c in zipped if l == c])

In [217]:
target_names = ['No space', 'Space', 'ZWNJ']
print(classification_report(y_true, y_pred, target_names=target_names))

             precision    recall  f1-score   support

   No space       0.99      0.99      0.99     59690
      Space       0.97      0.97      0.97     20316
       ZWNJ       0.90      0.79      0.84      1123

avg / total       0.98      0.98      0.98     81129

